<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Cookbook - LiteLLM (Proxy) + Langfuse OpenAI Integration (JS/TS)" sidebarTitle: "LiteLLM (Proxy) + Langfuse OpenAI Integration (JS/TS)" description: "The stack to use any of 100+ models in your JS/TS application without having to change your code and with full observability." category: "Examples" -->

# Cookbook: LiteLLM (Proxy) + Langfuse OpenAI Integration (JS/TS)

This notebook demonstrates how to use the following stack to experiment with 100+ LLMs from different providers without changing code:

- [**LiteLLM Proxy**](https://docs.litellm.ai/docs/) ([GitHub](https://github.com/BerriAI/litellm)): Standardizes 100+ model provider APIs on the OpenAI API schema.
- **Langfuse OpenAI SDK Wrapper** ([JS/TS](https://langfuse.com/integrations/model-providers/openai-js)): Natively instruments calls to 100+ models via the OpenAI SDK.
- **Langfuse**: OSS LLM Observability, full overview [here](https://langfuse.com/docs).

> **Note**: This cookbook uses **Deno.js** for execution, which requires different syntax for importing packages and setting environment variables. For Node.js applications, the setup process is similar but uses standard `npm` packages and `process.env`.

<!-- STEPS_START -->
## Set Up Environment

Get your Langfuse API keys by signing up for [Langfuse Cloud](https://cloud.langfuse.com/) or [self-hosting Langfuse](https://langfuse.com/self-hosting). You’ll also need your OpenAI API key.

> **Note**: This cookbook uses **Deno.js** for execution, which requires different syntax for importing packages and setting environment variables. For Node.js applications, the setup process is similar but uses standard `npm` packages and `process.env`.

In [ ]:
// Langfuse authentication keys
Deno.env.set("LANGFUSE_PUBLIC_KEY", "pk-lf-***");
Deno.env.set("LANGFUSE_SECRET_KEY", "sk-lf-***");

// Langfuse host configuration
// For US data region, set this to "https://us.cloud.langfuse.com"
Deno.env.set("LANGFUSE_HOST", "https://cloud.langfuse.com")

// Set environment variables using Deno-specific syntax
Deno.env.set("OPENAI_API_KEY", "sk-proj-***");

With the environment variables set, we can now initialize the `langfuseSpanProcessor` which is passed to the main OpenTelemetry SDK that orchestrates tracing.

In [ ]:
// Import required dependencies
import 'npm:dotenv/config';
import { NodeSDK } from "npm:@opentelemetry/sdk-node";
import { LangfuseSpanProcessor } from "npm:@langfuse/otel";
 
// Export the processor to be able to flush it later
// This is important for ensuring all spans are sent to Langfuse
export const langfuseSpanProcessor = new LangfuseSpanProcessor({
    publicKey: process.env.LANGFUSE_PUBLIC_KEY!,
    secretKey: process.env.LANGFUSE_SECRET_KEY!,
    baseUrl: process.env.LANGFUSE_HOST ?? 'https://cloud.langfuse.com', // Default to cloud if not specified
    environment: process.env.NODE_ENV ?? 'development', // Default to development if not specified
  });
 
// Initialize the OpenTelemetry SDK with our Langfuse processor
const sdk = new NodeSDK({
  spanProcessors: [langfuseSpanProcessor],
});
 
// Start the SDK to begin collecting telemetry
// The warning about crypto module is expected in Deno and doesn't affect basic tracing functionality. Media upload features will be disabled, but all core tracing works normally
sdk.start();

## Setup Lite LLM Proxy

In this example, we'll use llama3 via the Ollama on our local machine.

**Steps**

1. Create a `litellm_config.yaml` to configure which models are available ([docs](https://litellm.vercel.app/docs/proxy/configs)). We'll use llama3 via Ollama in this example. 
1. Ensure that you installed Ollama and have pulled the llama3 (8b) model: `ollama pull llama3`
2. Run the following cli command to start the proxy: `litellm --config litellm_config.yaml`

The Lite LLM Proxy should be now running on http://0.0.0.0:4000

To verify the connection you can run `litellm --test`

## Log single LLM Call via Langfuse OpenAI Wrapper

The Langfuse SDK offers a wrapper function around the OpenAI SDK, automatically logging all OpenAI calls as generations to Langfuse. We wrap the client for each call separately in order to be able to pass a name. For more details, please refer to our [documentation](https://langfuse.com/integrations/model-providers/openai-js).

In [ ]:
import { OpenAI } from "npm:openai";
import { observeOpenAI } from "npm:@langfuse/openai";

const PROXY_URL = "http://0.0.0.0:4000";
const client = observeOpenAI(new OpenAI({baseURL: PROXY_URL}));

const systemPrompt = "You are a very accurate calculator. You output only the result of the calculation.";

const llamaCompletion = await client.chat.completions.create({
  model: "ollama/llama3",
  messages: [
    {role: "system", content: systemPrompt},
    {role: "user", content: "3 + 3 = "}
  ],
}); 
console.log(llamaCompletion.choices[0].message.content);

6


[Public trace in the Langfuse UI](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/0b78e39b9a36195b7373a3c4536ad574?timestamp=2025-08-25T15%3A07%3A24.014Z&display=details&observation=fa33232a3cc957e1)

## Trace nested LLM Calls using Langfuse JS SDK

Use the context manager of the Langfuse TypeScript SDK to group multiple LiteLLM generations together and update the top level span.

In [ ]:
import { startActiveObservation, startObservation } from "npm:@langfuse/tracing";
import { observeOpenAI } from "npm:@langfuse/openai";

const client = observeOpenAI(new OpenAI({baseURL: PROXY_URL}));

const systemPrompt = "You are a very accurate calculator. You output only the result of the calculation.";
 
await startActiveObservation("user-request", async (span) => {

  await client.chat.completions.create({
    model: "ollama/llama3",
    messages: [
      {role: "system", content: systemPrompt},
      {role: "user", content: "3 + 3 = "}
    ],
  }); 

  await client.chat.completions.create({
    model: "ollama/llama3",
    messages: [
      {role: "system", content: systemPrompt},
      {role: "user", content: "1 - 1 = "}
    ],
  }); 

  await client.chat.completions.create({
    model: "ollama/llama3",
    messages: [
      {role: "system", content: systemPrompt},
      {role: "user", content: "2 + 3 = "}
    ],
  }); 

  // Update trace
  span.updateTrace({
    name:"LLM Calculator",
    tags: ["updated"],
    metadata: {"env": "development"},
    release: "v0.0.2",
    input: systemPrompt,
  });

});

## View traces in Langfuse

![Public Trace](https://langfuse.com/images/cookbook/example-js-sdk/integration_litellm_proxy_trace.png)

[Public trace in the Langfuse UI](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1aab1c70319ce53e2604801a5e464293?timestamp=2025-08-25T15%3A52%3A19.900Z&display=details&observation=b0f019915d463967)
<!-- STEPS_END -->

## Learn more

Check out the docs to learn more about all components of this stack:
- [LiteLLM Proxy](https://docs.litellm.ai/docs/)
- [Langfuse OpenAI SDK Wrapper](https://langfuse.com/integrations/model-providers/openai-js)
- [Langfuse](https://langfuse.com/docs)

If you do not want to capture traces via the OpenAI SDK Wrapper, you can also directly log requests from the LiteLLM Proxy to Langfuse. For more details, refer to the [LiteLLM Docs](https://litellm.vercel.app/docs/proxy/logging#logging-proxy-inputoutput---langfuse).